In [43]:
def helper(**kwargs):
    return f"helper_kwargs: {kwargs}"

def train_default_model(model_weights, model_architecture, **kwargs):
    return f'DEFAULT model: weights: {model_weights} \n architecture: {model_architecture}, \n default_kwargs: {helper(**kwargs)}'

def custom_train_model():
    return 'Trained CUSTOM model'

In [46]:
def create_training_function(train_model=custom_train_model, data_source='keras'):
    
    def training_function():
        result = train_model()
        
        if data_source == 'keras':
            return f"{data_source}: {result}"
        
        return result
    
    return training_function

In [48]:
cust_func = create_training_function(data_source='kek')
cust_func()

'Trained CUSTOM model'

In [25]:
train_default_model(123, 456, c=1, p=42)

"DEFAULT model: weights: 123 \n architecture: 456, \n default_kwargs: helper_kwargs: {'c': 1, 'p': 42}"

In [34]:
def get_new_data(num_samples = 100):
    from tensorflow import keras
    import numpy as np

    (x_train, y_train), _ = keras.datasets.mnist.load_data()
    
    # take a random set of images
    idx = np.random.choice(np.arange(len(x_train)), num_samples, replace=True)
    x_train = x_train[idx]
    y_train = y_train[idx]
    
    x_train = x_train.astype("float32") / 255
    x_train = np.expand_dims(x_train, -1)
    y_train = keras.utils.to_categorical(y_train, num_classes)

    return (x_train, y_train)

In [68]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

endpoint_ids = ['00929e1a-ccc5-40be-8b04-c171f132f7b2', '11983ca1-2d45-40d1-b5a2-8736b3544dea']
batch_size = 128
epochs = 5
input_shape = (28, 28, 1)
num_classes = 10

global_model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
    )

global_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# get the model's architecture and weights
json_config = global_model.to_json()
gm_weights = global_model.get_weights()
gm_weights_np = np.asarray(gm_weights, dtype=object)

In [61]:
(x_train, y_train) = get_new_data()

In [57]:
def train_default_model(json_model_config, 
                global_model_weights,
                x_train,
                y_train,
                epochs=10,
                loss="categorical_crossentropy",
                optimizer="adam", 
                metrics=["accuracy"],
                **extra_compiler_arguments):

    # import dependencies
    from tensorflow import keras

    # create the model
    model = keras.models.model_from_json(json_model_config)

    # compile the model and set weights to the global model
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics, **extra_compiler_arguments)
    model.set_weights(global_model_weights)

    # train the model on the local data and extract the weights
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)
    model_weights = model.get_weights()

    # transform to a numpy array
    np_model_weights = np.asarray(model_weights, dtype=object)

    return np_model_weights


train_default_model(json_config, gm_weights_np, x_train, y_train)

Epoch 1/10
1/1 [==============================] - 0s 2ms/step - loss: 2.3564 - accuracy: 0.0300
Epoch 2/10
1/1 [==============================] - 0s 2ms/step - loss: 2.3094 - accuracy: 0.1000
Epoch 3/10
1/1 [==============================] - 0s 2ms/step - loss: 2.2366 - accuracy: 0.2100
Epoch 4/10
1/1 [==============================] - 0s 2ms/step - loss: 2.2181 - accuracy: 0.2200
Epoch 5/10
1/1 [==============================] - 0s 1ms/step - loss: 2.1763 - accuracy: 0.3200
Epoch 6/10
1/1 [==============================] - 0s 1ms/step - loss: 2.1382 - accuracy: 0.3000
Epoch 7/10
1/1 [==============================] - 0s 2ms/step - loss: 2.1167 - accuracy: 0.3400
Epoch 8/10
1/1 [==============================] - 0s 2ms/step - loss: 2.0398 - accuracy: 0.4300
Epoch 9/10
1/1 [==============================] - 0s 1ms/step - loss: 2.0078 - accuracy: 0.4400
Epoch 10/10
1/1 [==============================] - 0s 2ms/step - loss: 1.9414 - accuracy: 0.4700


array([array([[[[-0.00308374, -0.02846969,  0.11923862,  0.1372091 ,
                  0.09622682, -0.12552725, -0.00572993,  0.13574612,
                  0.11874458,  0.00675087,  0.04690669,  0.03781084,
                  0.04851744,  0.13369915,  0.0447103 , -0.00706855,
                 -0.05891373,  0.08029041,  0.02478524, -0.06120633,
                 -0.12056763, -0.08738794, -0.10197891,  0.09668613,
                  0.0984856 ,  0.09276874,  0.11111507,  0.0628399 ,
                  0.10185242,  0.09389986, -0.01284155, -0.11243422]],

               [[-0.03743424,  0.14415571, -0.02378648, -0.04181101,
                  0.03629512,  0.00975019, -0.09322388,  0.05566316,
                 -0.05467005, -0.10323391, -0.12276282, -0.14051819,
                 -0.05226596, -0.06461635,  0.0919804 , -0.00729473,
                 -0.04277572,  0.07424589,  0.1187043 , -0.10935926,
                  0.03337073, -0.12946731, -0.0951025 ,  0.13218531,
                 -0.10267717,  

In [55]:
def train_default_model(json_model_config, 
                global_model_weights,
                x_train,
                y_train,
                epochs=10,
                **extra_compiler_arguments):

    # import dependencies
    from tensorflow import keras

    # create the model
    model = keras.models.model_from_json(json_model_config)

    # compile the model and set weights to the global model
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics, **extra_compiler_arguments)
    model.set_weights(global_model_weights)

    # train the model on the local data and extract the weights
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)
    model_weights = model.get_weights()

    # transform to a numpy array
    np_model_weights = np.asarray(model_weights, dtype=object)

    return np_model_weights




train_default_model(json_config, gm_weights_np, x_train, y_train)

NameError: name 'loss' is not defined

In [67]:
# conda activate py37
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

from funcx.sdk.client import FuncXClient
from funcx.sdk.executor import FuncXExecutor

# path_dir='/home/pi/datasets', x_train_path="mnist_x_train.npy", y_train_path="mnist_y_train.npy"
def get_local_data(x_train_path, y_train_path, path_dir=".", preprocess=None, preprocessing_function=None):
    '''
    Returns (x_train, y_train) given the edge directory and filenames.
    
    '''
    import numpy as np
    import os
    import collections
    
    x_train_path_file = os.sep.join([path_dir, x_train_path])
    y_train_path_file = os.sep.join([path_dir, y_train_path])

    with open(x_train_path_file, 'rb') as f:
        x_train = np.load(f)
        
    with open(y_train_path_file, 'rb') as f:
        y_train = np.load(f)

    if preprocess:
        # check if a valid function was given
        if not preprocessing_function or not isinstance(preprocessing_function, collections.Callable):
            raise TypeError('preprocessing_function is not a function. Please provide a valid function in your call')
        
        (x_train, y_train) = preprocessing_function(x_train, y_train)

    return (x_train, y_train)



def get_keras_data(keras_dataset, preprocess=True, num_samples=None):
    '''
    Returns (x_train, y_train) of a chosen built-in Keras dataset.
    Options: ['mnist', 'fashion_mnist', 'cifar10', 'cifar100', 'imdb', 'reuters', 'boston_housing']
    
    '''
    from tensorflow import keras
    import numpy as np

    available_datasets = ['mnist', 'fashion_mnist', 'cifar10', 'cifar100', 'imdb', 'reuters', 'boston_housing']
    dataset_mapping= {
        'mnist': keras.datasets.mnist,
        'fashion_mnist': keras.datasets.fashion_mnist,
        'cifar10': keras.datasets.cifar10,
        'cifar100': keras.datasets.cifar100,
        'imdb': keras.datasets.imdb,
        'reuters': keras.datasets.reuters,
        'boston_housing': keras.datasets.boston_housing
    }
    image_datasets = ['mnist', 'fashion_mnist', 'cifar10', 'cifar100']

    # check if the dataset exists
    if keras_dataset not in available_datasets:
        raise Exception(f"Please select one of the built-in Keras datasets: {available_datasets}")

    else:
        (x_train, y_train), _ = dataset_mapping[keras_dataset].load_data()

        # take a random set of images
        if num_samples:
            idx = np.random.choice(np.arange(len(x_train)), num_samples, replace=True)
            x_train = x_train[idx]
            y_train = y_train[idx]

        # do default image processing for built-in Keras images
        if preprocess:
            if keras_dataset in image_datasets:
                # Scale images to the [0, 1] range
                x_train = x_train.astype("float32") / 255

                # Make sure images have shape (num_samples, x, y, 1) if working with MNIST images
                if x_train.shape[-1] not in [1, 3]:
                    x_train = np.expand_dims(x_train, -1)

                # convert class vectors to binary class matrices
                num_classes=10
                y_train = keras.utils.to_categorical(y_train, num_classes)

        return (x_train, y_train)

def train_default_model(json_model_config, 
                global_model_weights,
                x_train,
                y_train,
                epochs=10,
                loss="categorical_crossentropy",
                optimizer="adam", 
                metrics=["accuracy"],
                **extra_compiler_arguments):

    # import dependencies
    from tensorflow import keras
    import numpy as np

    # create the model
    model = keras.models.model_from_json(json_model_config)

    # compile the model and set weights to the global model
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics, **extra_compiler_arguments)
    model.set_weights(global_model_weights)

    # train the model on the local data and extract the weights
    model.fit(x_train, y_train, epochs=epochs)
    model_weights = model.get_weights()

    # transform to a numpy array
    np_model_weights = np.asarray(model_weights, dtype=object)

    return np_model_weights


def create_training_function(train_model=train_default_model, 
                            data_source: str = "keras",
                            preprocessing_function=None,
                            path_dir='/home/pi/datasets', 
                            x_train_path="mnist_x_train.npy", 
                            y_train_path="mnist_y_train.npy", 
                            preprocess_local=True, 
                            keras_dataset = "mnist", 
                            preprocess_keras=True, 
                            loss="categorical_crossentropy",
                            optimizer="adam", 
                            metrics=["accuracy"],
                            get_keras_data=get_keras_data,
                            get_local_data=get_local_data,
                            **kwargs
):
    
    def training_function(json_model_config, 
                          global_model_weights, 
                          num_samples=None,
                          epochs=10,
                          **kwargs
):

        # import all the dependencies required for funcX functions)
        import numpy as np

        if data_source == 'local':
            (x_train, y_train) = get_local_data(path_dir=path_dir, 
                          x_train_path=x_train_path, 
                          y_train_path=y_train_path, 
                          preprocess=preprocess_local, 
                          preprocessing_function=preprocessing_function)

        elif data_source == 'keras':
            (x_train, y_train) = get_keras_data(keras_dataset, 
                                                preprocess_keras, 
                                                num_samples)

        else:
            raise Exception("Please choose one of data sources: ['local', 'keras']")

        # consider switching to args* and kwargs** in case of a custom training function
        model_weights = train_model(json_model_config, 
                                    global_model_weights, 
                                    x_train, 
                                    y_train,
                                    epochs,
                                    loss,
                                    optimizer, 
                                    metrics,
                                    **kwargs)

        return {"model_weights":model_weights, "samples_count": x_train.shape[0]}
    
    return training_function

def get_edge_weights(sample_counts):
    '''
    Returns weights for each model to find the weighted average 
    '''
    total = sum(sample_counts)
    fractions = sample_counts/total
    return fractions

def federated_average(global_model, 
                      endpoint_ids, 
                      num_samples=100,
                      epochs=10,
                      weighted=False,
                      train_model=train_default_model, 
                      data_source: str = "keras",
                      preprocessing_function=None,
                      path_dir='/home/pi/datasets', 
                      x_train_path="mnist_x_train.npy", 
                      y_train_path="mnist_y_train.npy", 
                      preprocess_local=None, 
                      keras_dataset = "mnist", 
                      preprocess_keras=True, 
                      loss="categorical_crossentropy",
                      optimizer="adam", 
                      metrics=["accuracy"],
                      **kwargs):

    fx = FuncXExecutor(FuncXClient())

    # get the model's architecture and weights
    json_config = global_model.to_json()
    gm_weights = global_model.get_weights()
    gm_weights_np = np.asarray(gm_weights, dtype=object)

    # compile the training function
    training_function = create_training_function(train_model=train_model, 
                                                data_source = data_source,
                                                path_dir=path_dir, 
                                                x_train_path=x_train_path, 
                                                y_train_path=y_train_path, 
                                                preprocess_local=preprocess_local, 
                                                preprocessing_function=preprocessing_function,
                                                keras_dataset = keras_dataset, 
                                                preprocess_keras=preprocess_keras, 
                                                loss=loss,
                                                optimizer=optimizer, 
                                                metrics=metrics)
    
    # train the MNIST model on each of the endpoints and return the result, sending the global weights to each edge
    tasks = []
    for e in endpoint_ids:
        tasks.append(fx.submit(training_function, 
                                json_model_config=json_config, 
                                global_model_weights=gm_weights_np, 
                                num_samples=num_samples,
                                epochs=epochs,
                                endpoint_id=e))
    
    # extract weights from each edge model
    model_weights = [t.result()["model_weights"] for t in tasks]
    
    if weighted:
        # get the weights
        sample_counts = np.array([t.result()["samples_count"] for t in tasks])
        edge_weights = get_edge_weights(sample_counts)
        
        print(f"Model Weights: {edge_weights}")
        # find weighted average
        average_weights = np.average(model_weights, weights=edge_weights, axis=0)
        
    else:
        # simple average of the weights
        average_weights = np.mean(model_weights, axis=0)
    
    # assign the weights to the global_model
    global_model.set_weights(average_weights)

    print('Trained Federated Model')

    return global_model



In [69]:
# get the model's architecture and weights
json_config = global_model.to_json()
gm_weights = global_model.get_weights()
gm_weights_np = np.asarray(gm_weights, dtype=object)

# compile the training function
training_function = create_training_function()

In [71]:
num_samples=100,
epochs=10
weighted=False
train_model=train_default_model,
data_source="keras"
preprocessing_function=None
path_dir='/home/pi/datasets'
x_train_path="mnist_x_train.npy"
y_train_path="mnist_y_train.npy"
preprocess_local=None
keras_dataset = "mnist"
preprocess_keras=True
loss="categorical_crossentropy"
optimizer="adam"
metrics=["accuracy"]

training_function = create_training_function(train_model=train_model, 
                                            data_source = data_source,
                                            path_dir=path_dir, 
                                            x_train_path=x_train_path, 
                                            y_train_path=y_train_path, 
                                            preprocess_local=preprocess_local, 
                                            preprocessing_function=preprocessing_function,
                                            keras_dataset = keras_dataset, 
                                            preprocess_keras=preprocess_keras, 
                                            loss=loss,
                                            optimizer=optimizer, 
                                            metrics=metrics)

In [73]:
training_function(json_model_config=json_config, global_model_weights=gm_weights_np)

TypeError: 'tuple' object is not callable

In [74]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

endpoint_ids = ['00929e1a-ccc5-40be-8b04-c171f132f7b2', '11983ca1-2d45-40d1-b5a2-8736b3544dea']
batch_size = 128
epochs = 5
input_shape = (28, 28, 1)
num_classes = 10

global_model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
    )

global_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [75]:
def get_test_data(num_samples=1000):
    from tensorflow import keras
    num_classes = 10
    _, (x_test, y_test) = keras.datasets.mnist.load_data()
    
    idx = np.random.choice(np.arange(len(x_test)), num_samples, replace=True)
    x_test = x_test[idx]
    y_test = y_test[idx]
    
    x_test = x_test.astype("float32") / 255
    x_test = np.expand_dims(x_test, -1)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    return (x_test, y_test)

def eval_model(m, x, y):
    ''' evaluate model on dataset x,y'''
    score = m.evaluate(x, y, verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])
    
x_test, y_test = get_test_data()

In [76]:
def preprocess_data(x_train, y_train, num_samples=100):
    from tensorflow import keras
    import numpy as np

    num_classes = 10

    # take a random set of images
    idx = np.random.choice(np.arange(len(x_train)), num_samples, replace=True)
    x_train = x_train[idx]
    y_train = y_train[idx]

    # Scale images to the [0, 1] range
    x_train = x_train.astype("float32") / 255

    # Make sure images have shape (28, 28, 1)
    x_train = np.expand_dims(x_train, -1)
    print("x_train shape:", x_train.shape)
    print(x_train.shape[0], "train samples")

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)

    return (x_train, y_train)

In [ ]:
for i in range(2):
    federated_average(global_model=global_model, 
                  endpoint_ids=endpoint_ids,
                  data_source="local",
                  path_dir='/home/pi/datasets', 
                  x_train_path="mnist_x_train.npy", 
                  y_train_path="mnist_y_train.npy", 
                  preprocess_local=True,
                  preprocessing_function=preprocess_data)
    eval_model(global_model, x_test, y_test)